In [1]:
pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0 

Note: you may need to restart the kernel to use updated packages.


In [18]:
import mlflow
import mlflow.sagemaker
from mlflow.tracking import MlflowClient
import numpy as np
from sagemaker.serve import SchemaBuilder
from sagemaker.serve import ModelBuilder
from sagemaker import get_execution_role
from sagemaker.serve.mode.function_pointers import Mode

In [3]:
mlflow_arn = "arn:aws:sagemaker:eu-west-1:211125740051:mlflow-tracking-server/Sample-server"
registered_model_name = "wine-model"

In [16]:
# Set the MLflow tracking URI (you may need to set this if not already configured)
mlflow.set_tracking_uri(mlflow_arn)
client = MlflowClient()
role = get_execution_role()

In [6]:
registered_model = client.get_registered_model(name=registered_model_name)
source_path = registered_model.latest_versions[0].source

In [13]:
sklearn_input = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]).reshape(1, -1)
sklearn_output = 1.0
sklearn_schema_builder = SchemaBuilder(
    sample_input=sklearn_input,
    sample_output=sklearn_output,
)

In [19]:
# Create model builder with the schema builder.
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=sklearn_schema_builder,
    role_arn=role,
    model_metadata={
        "MLFLOW_MODEL_PATH": source_path
    }
)

In [20]:
built_model = model_builder.build()

ModelBuilder: WARNING:     Support of MLflow format models is experimental and is not intended for production at this moment.


ModelBuilder: INFO:     Auto-detected deployment flavor is sklearn
ModelBuilder: INFO:     Auto-detected framework version is 1.4.2


ModelBuilder: INFO:     Auto detected 141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3. Proceeding with the the deployment.
ModelBuilder: WARNING:     Image 141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3 uses python version 3.8. Your python version used for model is 3.10.14. It is recommended to use the same python version to avoid incompatibility.
ModelBuilder: INFO:     Skipping auto detection as the image uri is provided 141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
ModelBuilder: DEBUG:     Packaging provided requirements.txt from /tmp/sagemaker/model-builder/6c2ef19a33a011efbba2e600116ddf77/requirements.txt
ModelBuilder: DEBUG:     Uploading the model resources to bucket=sagemaker-eu-west-1-211125740051, key_prefix=sagemaker-scikit-learn-2024-06-26-09-44-45-180.
Uploading model artifacts: 100%|████████████████████████| 13725/13725 [00:00<00:00, 70754.46bytes/s]
ModelBuilder: DEBU

In [27]:
predictor = built_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-06-26-09-47-13-455
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-06-26-09-47-14-236
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-06-26-09-47-14-236


----------------------------------------------*

ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


UnexpectedStatusException: Error hosting endpoint sagemaker-scikit-learn-2024-06-26-09-47-14-236: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html